In [3]:
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'iTf***',
    'service_id': 'iam-ServiceId-d4b06e46-293a-4417-b76c-2f16076a9353',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_b0f1407510994fd1b793b85137baafb8_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df = spark.read.parquet(cos.url('hmp.parquet', 'courseraml-donotdelete-pr-qve0ttzezdeodc'))
df.show()


+---+---+---+--------------------+------------+
|  x|  y|  z|              source|       class|
+---+---+---+--------------------+------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|
| 32| 33| 36|Accelerometer-201...|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|
| 30| 37| 50|Accelerometer-201...|Climb_stairs|
| 31| 37| 50|Accelerometer-201...|Climb_stairs|
| 29| 38| 50|Accelerometer-201...|Climb_stairs|
| 32| 39| 48|Accelerometer-201...|Climb_stairs|
| 32| 41| 46|Accelerometer-201...|Climb_stairs|
| 34| 39| 44|Accelerometer-201...|Climb_stairs|
| 34| 39| 41|Accelerometer-201...|Climb_stairs|
| 31| 39| 42|Accelerometer-201...|Climb_stairs|
| 32| 42| 43|Accelerometer-201...|Climb_stairs|
| 31| 43| 43|Accelerometer-201...|Climb_stairs|
| 28| 46| 44|Accelerometer-201...|Climb_

In [5]:
df.createOrReplaceTempView('df')

# linear regresssion
df_energy = spark.sql("""
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class
""")
df_energy.show()
df_energy.createOrReplaceTempView('df_energy')

+------------------+--------------+
|             label|         class|
+------------------+--------------+
| 8959.680239829991| Use_telephone|
| 9737.511232342687| Standup_chair|
| 12542.96539897962|      Eat_meat|
|13225.945637269193|     Getup_bed|
|15003.269043778426|   Drink_glass|
|14454.885091207056|    Pour_water|
|10616.408809008817|     Comb_hair|
|11082.626493751379|  Climb_stairs|
|10261.338314274606| Sitdown_chair|
|6783.4063714331605|   Liedown_bed|
| 7173.493500380411|Descend_stairs|
| 11785.39634462923|   Brush_teeth|
| 6071.460120926432|      Eat_soup|
+------------------+--------------+



In [6]:
df_join = spark.sql("""
select * from df inner join df_energy on df.class=df_energy.class
""")
df_join.show()

+---+---+---+--------------------+------------+------------------+------------+
|  x|  y|  z|              source|       class|             label|       class|
+---+---+---+--------------------+------------+------------------+------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 33| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 30| 37| 50|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 31| 37| 50|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 29| 38| 50|Accelerometer-201...|Climb_

In [7]:
from pyspark.ml.feature import VectorAssembler, Normalizer

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

In [8]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [9]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer,lr])

In [10]:
model = pipeline.fit(df_join)
prediction = model.transform(df_join)

prediction.show()

+---+---+---+--------------------+------------+------------------+------------+----------------+--------------------+------------------+
|  x|  y|  z|              source|       class|             label|       class|        features|       features_norm|        prediction|
+---+---+---+--------------------+------------+------------------+------------+----------------+--------------------+------------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[30.0,36.0,52.0]|[0.25423728813559...|12378.337524478058|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[30.0,36.0,32.0]|[0.30612244897959...|10949.223755865942|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[32.0,30.0,36.0]|[0.32653061224489...|11232.049623881609|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[32.0,30.0,36.0]|[0.32653061224489...|11232.049623881609|
| 32| 30| 36|Accelerometer-201...|Climb_s

In [12]:
model.stages[2].summary.r2

0.0912330479847846